# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686559


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:03<00:47,  1.63s/it]

Rendering models:  10%|▉         | 3/31 [00:03<00:33,  1.20s/it]

Rendering models:  13%|█▎        | 4/31 [00:08<00:59,  2.21s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:44,  1.72s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:19<00:01,  1.93it/s]

Rendering models:  94%|█████████▎| 29/31 [00:19<00:01,  1.69it/s]

Rendering models:  97%|█████████▋| 30/31 [00:20<00:00,  1.84it/s]

Rendering models: 100%|██████████| 31/31 [00:20<00:00,  1.97it/s]

not-logged-in-3e461b5a5635ce55ae0f     0.001710
MissDonkey                             0.000808
not-logged-in-3b85ef6ad3e2968c6631     0.001403
not-logged-in-20a275b53abec8db9f41     0.322409
smithkurtis                            0.000702
CThomas                                0.000743
Dynamatt                               0.000562
mkehrli                                0.000528
shocko61                               0.000682
djswanso                               0.000575
shocko61                               0.000790
Kamphuisjes                            0.000605
not-logged-in-4447ff92852aa7c13340     0.000663
sn346808                               0.023179
not-logged-in-116eef09a8f6e73ef4da     0.000886
LeeJoseph                              0.005189
Sylverone                              0.000577
ranyel.a.e.pasimio                     0.627675
ELight                                23.016239
not-logged-in-e47d32a1e7d361d3ca0e     0.160760
DylanGonzalez                          0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())